# LLM（大規模言語モデル）のファインチューニングの例

CyberAgentが公開している Open-Calm と呼ばれるLLM（大規模言語モデル）をファインチューニングして問題を解いてみます．

[ライブドアニュースコーパス](https://www.rondhuit.com/download.html#news%20corpus)を使って，ニュース記事のタイトルからカテゴリを予測するモデルを作成します．

参考: https://note.com/npaka/n/na5b8e6f749ce


## ライブドアニュースのダウンロードと展開（必要に応じて）

In [1]:
import os
import urllib.request
import tarfile

# textディレクトリが存在しない場合
if not os.path.exists('text'):
    # ldcc-20140209.tar.gzファイルが存在しない場合
    if not os.path.exists('ldcc-20140209.tar.gz'):
        # ダウンロード
        url = 'https://www.rondhuit.com/download/ldcc-20140209.tar.gz'
        urllib.request.urlretrieve(url, 'ldcc-20140209.tar.gz')
    
    # tarファイルの展開
    with tarfile.open('ldcc-20140209.tar.gz', 'r:gz') as tar:
        tar.extractall()

## 事前学習モデルの読み込み

In [2]:
# 基本パラメータ
model_name = "cyberagent/open-calm-medium"   # モデルの名前
peft_name  = "lora-calm-medium"              # 学習用モデル(PEFTモデル)の名前
output_dir = "lora-calm-medium-results"      # 学習結果の出力先

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(model_name)

## データセットの作成

In [4]:
# トークナイザのスペシャルトークンの確認
print(tokenizer.special_tokens_map)
print("bos_token :", tokenizer.eos_token, ",", tokenizer.bos_token_id)
print("eos_token :", tokenizer.bos_token, ",", tokenizer.eos_token_id)
print("unk_token :", tokenizer.unk_token, ",", tokenizer.unk_token_id)
print("pad_token :", tokenizer.pad_token, ",", tokenizer.pad_token_id)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|padding|>'}
bos_token : <|endoftext|> , 0
eos_token : <|endoftext|> , 0
unk_token : <|endoftext|> , 0
pad_token : <|padding|> , 1


In [5]:
CUTOFF_LEN = 512  # 最大長

# トークナイズ関数の定義
def tokenize(prompt, tokenizer):
    result = tokenizer(
        prompt+"<|endoftext|>",  # EOSの付加
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=True,
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

In [6]:
# トークナイズの動作確認
tokenize("私の名前はシェーマです", tokenizer)

{'input_ids': [984, 18504, 2281, 1114, 358, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [7]:
import os
import glob

dirname2label = {
    'dokujo-tsushin': "独女通信",
    'it-life-hack': "ITライフハック",
    'kaden-channel': "家電チャンネル",
    'livedoor-homme': "livedoor HOMME",
    'movie-enter': "MOVIE ENTER",
    'peachy': "Peachy",
    'smax': "エスマックス",
    'sports-watch': "Sports Watch",
    'topic-news': "トピックニュース",
}

data_all = []
for dirname, label in dirname2label.items():
    os.path.join('text', dirname)
    filepaths = glob.glob(os.path.join('text', dirname, '*.txt'))
    for filepath in filepaths:
        text = open(filepath, 'r', encoding='utf-8').read()
        # 内容のある最初の３文だけ取り出す
        lines = []
        for line in text.split('\n')[2:]:
            if len(line) > 0:
                lines.append(line)
                if len(lines) >= 3:
                    break
        text = '\n'.join(lines) + '\n'
        data_all.append({'text': text, 'response': label})

In [8]:
print(data_all[0]['text'])

森ガールよりもリア充？ 「雲ガール」の実態に迫る
秋深き　隣は何を　する人ぞ　——。これまで「森ガール」にはじまり「山ガール」、「釣りガール」とさまざまな流行を生みだしてきた“○○ガールズ”ブーム。2011年も本格的に秋の様相を呈してきた今、新たなガールズブームが産声を上げた。その名も「雲ガール」。これはPCやスマートフォンでインターネットを活用し、趣味や生活を楽しんでいる女子たちのこと。ちなみに「雲」は、今もっともホットなインターネットサービス「クラウド」を指すのである。
今回、この雲ガールたちの座談会があるとの情報をキャッチ！　潜入し、雲ガールの実態をチェックしてきたので、ここに報告しよう。



In [9]:
print(data_all[0]['response'])

独女通信


In [10]:
# プロンプト生成（正解データ）のための関数
def generate_prompt(data_point):
    return f"""{data_point['text']}

### Label:
{data_point['response']}"""

In [11]:
print(generate_prompt(data_all[0]))

森ガールよりもリア充？ 「雲ガール」の実態に迫る
秋深き　隣は何を　する人ぞ　——。これまで「森ガール」にはじまり「山ガール」、「釣りガール」とさまざまな流行を生みだしてきた“○○ガールズ”ブーム。2011年も本格的に秋の様相を呈してきた今、新たなガールズブームが産声を上げた。その名も「雲ガール」。これはPCやスマートフォンでインターネットを活用し、趣味や生活を楽しんでいる女子たちのこと。ちなみに「雲」は、今もっともホットなインターネットサービス「クラウド」を指すのである。
今回、この雲ガールたちの座談会があるとの情報をキャッチ！　潜入し、雲ガールの実態をチェックしてきたので、ここに報告しよう。


### Label:
独女通信


In [12]:
from datasets import Dataset
import pandas as pd

# dictionary -> pandas dataframe -> datasets.Dataset
df_data_all = pd.DataFrame(data_all)
dataset_all = Dataset.from_pandas(df_data_all)

# 検証データの数
VAL_SET_SIZE = 200

# 学習データと検証データに分割
train_val = dataset_all.train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]

# プロンプトへの変換とトークナイズ
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))

## 学習用モデルの準備

In [13]:
from transformers import AutoModelForCausalLM

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/fujie/.conda/envs/py39 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# LoRAのパラメータ
lora_config = LoraConfig(
    r= 8, 
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# モデルの前処理
model = prepare_model_for_int8_training(model)

# LoRAモデルの準備
model = get_peft_model(model, lora_config)

# 学習可能パラメータの確認
model.print_trainable_parameters()

trainable params: 786432 || all params: 409790464 || trainable%: 0.19191076149590441


In [15]:
import transformers
eval_steps = 200
save_steps = 200
logging_steps = 20

# トレーナーの準備
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        num_train_epochs=10,
        learning_rate=3e-4,
        logging_steps=logging_steps,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=output_dir,
        report_to="none",
        save_total_limit=3,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [16]:
# 学習の実行
model.config.use_cache = False
trainer.train() 
model.config.use_cache = True

# LoRAモデルの保存
trainer.model.save_pretrained(peft_name)

/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:230: UserWarning: where received a uint8 co

Step,Training Loss,Validation Loss
200,2.949500,2.907751
400,2.874000,2.867150
600,2.878200,2.839062
800,2.845800,2.823074
1000,2.814000,2.808615
1200,2.759700,2.799341
1400,2.827600,2.786136
1600,2.787400,2.779655
1800,2.771500,2.766935
2000,2.749900,2.761552


/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatM

In [10]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# トークンナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRAモデルの準備
model = PeftModel.from_pretrained(
    model, 
    peft_name, 
    device_map="auto"
)

# 評価モード
model.eval()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/fujie/.conda/envs/py39 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(52096, 1024)
        (layers): ModuleList(
          (0-23): 24 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=1024, out_features=3072, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                

In [11]:
def generate_query_prompt(data_point):
    return f"""{data_point['text']}

### Label:
"""

In [12]:
# テキスト生成関数の定義
def generate(instruction,input=None,maxTokens=512):
    # 推論
    prompt = generate_query_prompt({'text':instruction})
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids, 
        max_new_tokens=maxTokens, 
        do_sample=True,
        temperature=0.7, 
        top_p=0.75, 
        top_k=40,         
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # ラベル内容のみ抽出
        sentinel = "### Label:"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            response = decoded[sentinelLoc+len(sentinel):]
            # print(response)
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
            response = None
    else:
        print('Warning: no <eos> detected ignoring output')
        response = None
        decoded = None

    return response, prompt, decoded

In [25]:
# d = val_data[33]
d = data_all[1700]

response, prompt, decoded = generate(d['text'])
print("---- PROMPT ----")
print(prompt)
print("---- RESPONSE ----")
print(response)
print("---- TARGET ----")
print(d['response'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


---- PROMPT ----
ウルトラブックとは一体何なのか？パソコン市場は縮小するのか【デジ通】
2011年の6月にそのコンセプトが発表されたUltrabook(ウルトラブック)だが、2011年末から2012年初頭にかけて、Ultrabook第1弾と呼べる製品が多数登場し、市場でも一定の人気を得ているようだ。
今後も新製品が多数予定されているが、Ultrabookの本命は2013年にインテルから登場する新プロセッサHaswell(コード名)を採用する製品と言われており、それまでにどう成長していくか注目される。


### Label:

---- RESPONSE ----

ITライフ!業界ニュース
---- TARGET ----
ITライフハック


In [16]:
d = dict(text = """藻類から燃料、CO2実質ゼロ　ちとせ研究所が大型施設
温暖化ガスの排出が実質ゼロの燃料を生物の力で作る取り組みが進む。
ちとせ研究所（川崎市）は4月、マレーシアに二酸化炭素（CO2）から燃料の原料を作る藻類を培養する世界最大規模の施設を開設した。
将来的に製造コストを化石燃料と競争できる水準に抑えることを目指す。
""",
response = "unknown")

response, prompt, decoded = generate(d['text'])
print("---- PROMPT ----")
print(prompt)
print("---- RESPONSE ----")
print(response)
print("---- TARGET ----")
print(d['response'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


---- PROMPT ----
藻類から燃料、CO2実質ゼロ　ちとせ研究所が大型施設
温暖化ガスの排出が実質ゼロの燃料を生物の力で作る取り組みが進む。
ちとせ研究所（川崎市）は4月、マレーシアに二酸化炭素（CO2）から燃料の原料を作る藻類を培養する世界最大規模の施設を開設した。
将来的に製造コストを化石燃料と競争できる水準に抑えることを目指す。


### Label:

---- RESPONSE ----

トピックニュース
---- TARGET ----
unknown


In [14]:
d = dict(text = """アラサーの乾燥におすすめの化粧水ランキング8選！プチプラからデパコスまで紹介
「アラサー乾燥肌におすすめの化粧水はどれ？」とお悩みではありませんか？
そこで今回は、アラサー女子向けで乾燥になる方向けにおすすめ化粧水を紹介します。
ドラッグストアなどで買えるプチプラ化粧水からデパコス化粧水まで幅広く解説！
""",
response = "unknown")

response, prompt, decoded = generate(d['text'])
print("---- PROMPT ----")
print(prompt)
print("---- RESPONSE ----")
print(response)
print("---- TARGET ----")
print(d['response'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:230: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  attn_scores = torch.where(causal_mask, attn_scores, mask_value)


---- PROMPT ----
アラサーの乾燥におすすめの化粧水ランキング8選！プチプラからデパコスまで紹介
「アラサー乾燥肌におすすめの化粧水はどれ？」とお悩みではありませんか？
そこで今回は、アラサー女子向けで乾燥になる方向けにおすすめ化粧水を紹介します。
ドラッグストアなどで買えるプチプラ化粧水からデパコス化粧水まで幅広く解説！


### Label:

---- RESPONSE ----

Peachy
---- TARGET ----
unknown
